<a href="https://colab.research.google.com/github/vina12alvina/Homework_MapReduce/blob/main/Hadoop_Hands_On_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Hadoop is an open-source framework which is mainly used for storage purposes and maintaining and analyzing a large amount of data or datasets on the clusters of commodity hardware, which means it is actually a data management tool.

##Hadoop mainly works on 3 different modes:

###Standalone Mode

Pseudo-distributed Mode
Fully-distributed Mode
Standalone Mode

By default, Hadoop is configured to run in a non distributed mode. It runs as a single Java process. Instead of HDFS, this mode utilizes the local file system. This mode is useful for debugging and there isn't any need to configure core-site.xml, hdfs-site.xml, mapred-site.xml, masters & slaves. Stand-alone mode is usually the fastest mode in Hadoop.

###Pseudo-distributed Mode

Hadoop can also run on a single node in a Pseudo-distributed mode. In this mode, each daemon runs on separate java processes. In this mode custom configuration is required (core-site.xml, hdfs-site.xml, mapred-site.xml). Here HDFS is utilized for input and output. This mode of deployment is useful for testing and debugging purposes.

###Fully-distributed Mode

This is the production mode of Hadoop. In this mode typically one machine in the cluster is designated as NameNode and another as Resource Manager exclusively. These are masters. All other nodes act as Data Node and Node Manager. These are the slaves. Configuration parameters and environment need to be specified for Hadoop Daemons.

Installing Java 8
Hadoop is a java programming-based data processing framework

OpenJDK is a development environment for building applications, applets, and components using the Java programming language.

# Installing Java 8
Hadoop is a java programming-based data processing framework

OpenJDK is a development environment for building applications, applets, and components using the Java programming language.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!update-alternatives --set jps /usr/lib/jvm/java-8-openjdk-amd64/bin/jps
!java -version

#Finding the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"
!apt-get install openssh-server -qq > /dev/null
!service ssh start

!grep Port /etc/ssh/sshd_config

#Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa

# See id_rsa.pub content
!more /root/.ssh/id_rsa.pub

#Copying the key to autorized keys
!cat $HOME/.ssh/id_rsa.pub > $HOME/.ssh/authorized_keys
#Changing the permissions on the key
!chmod 0600 ~/.ssh/authorized_keys

#Conneting with the local machine
!ssh -o StrictHostKeyChecking=no localhost uptime


#Downloading Hadoop 3.2.3
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz

#Untarring the file
!sudo tar -xzf hadoop-3.2.3.tar.gz
#Removing the tar file
!rm hadoop-3.2.3.tar.gz


#Copying the hadoop files to user/local
!cp -r hadoop-3.2.3/ /usr/local/
#-r copy directories recursively

#Adding JAVA_HOME directory to hadoop-env.sh file
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.3/etc/hadoop/hadoop-env.sh

import os
#Creating environment variables
#Creating Hadoop home variable

os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.3"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += f'{os.environ["JAVA_HOME"]}/bin:{os.environ["JRE_HOME"]}/bin:{os.environ["HADOOP_HOME"]}/sbin'

#Dowloading text example to use as input
# !wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javac to provide /usr/bin/javac (javac) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jps to provide /usr/bin/jps (jps) in manual mode
openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-8u402-ga-2ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64/jre/
 * Starting OpenBSD Secure Shell server sshd
   ...done.
#Port 22
#GatewayPorts no
Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in 

In [2]:
#Adding required property to core-site.xlm file
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>
EOF

In [3]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>

</configuration>
EOF

In [4]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.application.classpath</name>
    <value>$HADOOP_HOME/share/hadoop/mapreduce/*:$HADOOP_HOME/share/hadoop/mapreduce/lib/*</value>
  </property>

</configuration>
EOF

In [5]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
<property>
    <description>The hostname of the RM.</description>
    <name>yarn.resourcemanager.hostname</name>
    <value>localhost</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.nodemanager.env-whitelist</name>
    <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_HOME,PATH,LANG,TZ,HADOOP_MAPRED_HOME</value>
  </property>

<!-- Site specific YARN configuration properties -->

</configuration>
EOF

# Formatting the HDFS Filesystem

Before HDFS can be used for the first time the file system must be formatted. The formatting process creates an empty file system by creating the storage directories and the initial versions of the NameNodes

In [6]:
!$HADOOP_HOME/bin/hdfs namenode -format

#Creating other necessary enviroment variables before starting nodes
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

#Launching hdfs deamons
!$HADOOP_HOME/sbin/start-dfs.sh

#Launching yarn deamons
#nohup causes a process to ignore a SIGHUP signal
!nohup $HADOOP_HOME/sbin/start-yarn.sh

#Listing the running deamons
!jps

2024-05-04 07:37:19,418 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = e5bfe77515ff/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.3
STARTUP_MSG:   classpath = /usr/local/hadoop-3.2.3/etc/hadoop:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jsr305-3.0.2.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-net-3.6.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/javax.servlet-api-3.1.0.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/kerb-simplekdc-1.0.1.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jackson-core-asl-1.9.13.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/metrics-core-3.2.4.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jaxb-api-2.2.11.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/kerby-config-1.0.1.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-compress-1.21.jar:/usr/local/hadoop-3.2.3/share/h

### Monitoring Hadoop cluster with hadoop admin commands

In [7]:
#Report the basic file system information and statistics
!$HADOOP_HOME/bin/hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 85084090368 (79.24 GB)
DFS Remaining: 85084065792 (79.24 GB)
DFS Used: 24576 (24 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: e5bfe77515ff
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 30557323264 (28.46 GB)
DFS Remaining: 85084065792 (79.24 GB)
DFS Used%: 0.00%
DFS Remaining%: 73.57%
Configured Cache Capacity: 0 (0 B)
Cache

# MapReduce

In [ ]:
while True: pass

KeyboardInterrupt: 

In [8]:
#Dowloading text example to use as input (if it has not been donwloaded yet)
!wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count
#Coping file from local file system to HDFS
!$HADOOP_HOME/bin/hdfs dfs -put /content/101.txt /word_count

#Exploring Hadoop folder
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count

# Run MapReduce Example using JAVA
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar wordcount /word_count/101.txt /word_count/output/

Found 1 items
-rw-r--r--   1 root supergroup     678064 2024-05-04 07:38 /word_count/101.txt
2024-05-04 07:38:22,360 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-04 07:38:23,438 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1714808272884_0001
2024-05-04 07:38:24,046 INFO input.FileInputFormat: Total input files to process : 1
2024-05-04 07:38:24,664 INFO mapreduce.JobSubmitter: number of splits:1
2024-05-04 07:38:24,884 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1714808272884_0001
2024-05-04 07:38:24,886 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-05-04 07:38:25,186 INFO conf.Configuration: resource-types.xml not found
2024-05-04 07:38:25,186 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-05-04 07:38:25,721 INFO impl.YarnClientImpl: Submitted application application_1714808272884_0001
2024-05-04 07:38:25,789 INFO mapreduc

In [9]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-05-04 07:38 /word_count/output/_SUCCESS
-rw-r--r--   1 root supergroup     233636 2024-05-04 07:38 /word_count/output/part-r-00000


In [10]:
#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count/output/part-r-00000 | head -50

"(d)	1
"/H"	1
"0"	1
"02"	2
"1.	1
"414	3
"A	2
"AT&T	3
"AT&T's	1
"Access	1
"Acid	3
"Ad-hocracy"	1
"Advanced	1
"Agents	1
"Al	3
"All	3
"American	1
"An	1
"And	2
"Any	1
"Are	3
"Artificial	1
"As	1
"Assistant	1
"Attctc"	2
"Auld	1
"Autodesk,"	1
"BBS,"	2
"BIRTHPLACE	1
"BRITS	1
"Barry	1
"Because	1
"Before	1
"Bell	2
"Bell"	1
"BellSouth	2
"Berkeley	1
"Big	1
"Biggest	1
"Black	3
"Blue	1
"Bob"	1
"Bob,	3
"Bob:	1
"Bullet-N-Board."	1
"Bureaucrat-ese	1
"But	4
"C-word."	1
"CALIFORNIA"	1
"CC,"	1
cat: Unable to write to output stream.


# Hadoop Streaming Using Python

Hadoop Streaming is a feature that comes with Hadoop and allows users or developers to use various different languages for writing MapReduce programs like Python, C++, Ruby, etc.

The utility will create a Map/Reduce job, submit the job to an appropriate cluster, and monitor the progress of the job until it completes.

In [11]:
#Exploring Hadoop utilities available
!ls $HADOOP_HOME/share/hadoop/tools/lib/

aliyun-java-sdk-core-4.5.10.jar      hadoop-gridmix-3.2.3.jar
aliyun-java-sdk-kms-2.11.0.jar	     hadoop-kafka-3.2.3.jar
aliyun-java-sdk-ram-3.1.0.jar	     hadoop-openstack-3.2.3.jar
aliyun-sdk-oss-3.13.0.jar	     hadoop-resourceestimator-3.2.3.jar
aws-java-sdk-bundle-1.11.901.jar     hadoop-rumen-3.2.3.jar
azure-data-lake-store-sdk-2.2.9.jar  hadoop-sls-3.2.3.jar
azure-keyvault-core-1.0.0.jar	     hadoop-streaming-3.2.3.jar
azure-storage-7.0.0.jar		     ini4j-0.5.4.jar
hadoop-aliyun-3.2.3.jar		     jdom2-2.0.6.jar
hadoop-archive-logs-3.2.3.jar	     kafka-clients-2.8.1.jar
hadoop-archives-3.2.3.jar	     lz4-java-1.7.1.jar
hadoop-aws-3.2.3.jar		     ojalgo-43.0.jar
hadoop-azure-3.2.3.jar		     opentracing-api-0.33.0.jar
hadoop-azure-datalake-3.2.3.jar      opentracing-noop-0.33.0.jar
hadoop-datajoin-3.2.3.jar	     opentracing-util-0.33.0.jar
hadoop-distcp-3.2.3.jar		     org.jacoco.agent-0.8.5-runtime.jar
hadoop-extras-3.2.3.jar		     wildfly-openssl-1.0.7.Final.jar
hadoop-fs2img-3.2.3.

In [12]:
#Dowloading text example to use as input (if it has not been donwloaded yet)
!wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

In [13]:
#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count_with_python

In [14]:
#Copying the file from local file system to Hadoop distributed file system (HDFS)
!$HADOOP_HOME/bin/hdfs dfs -put /content/101.txt /word_count_with_python

## Mapper

The mapper is an executable that reads all input records from a file/s and generates an output in the form of key-value pairs which works as input for the Reducer.

In [15]:
%%writefile mapper.py

#!/usr/bin/env python

#'#!' is known as shebang and used for interpreting the script

# import sys because we need to read and write data to STDIN and STDOUT
import sys

# reading entire line from STDIN (standard input)
for line in sys.stdin:
  # to remove leading and trailing whitespace
  line = line.strip()
  # split the line into words, output data type list
  words = line.split()

  # we are looping over the words array and printing the word
  # with the count of 1 to the STDOUT
  for word in words:
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s\t%s' % (word, 1))

Writing mapper.py


In [16]:
line = "Hi, I'm Bruce Sterling, the author of this electronic book. "
line = line.strip()
words = line.split()
for word in words:
  data = '%s\t%s' % (word, 1)

  word, count = data.split('\t', 1)
  print(word, count)


Hi, 1
I'm 1
Bruce 1
Sterling, 1
the 1
author 1
of 1
this 1
electronic 1
book. 1


## Reducer

The reducer is an executable that reads all the intermediate key-value pairs generated by the mapper and generates a final output as a result of a computation operation like addition, filtration, and aggregation.

Both the mapper and the reducer read the input from stdin (line by line) and emit the output to stdout.

In [17]:
%%writefile reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# read the entire line from STDIN
for line in sys.stdin:
  # remove leading and trailing whitespace
  line = line.strip()
  # splitting the data on the basis of tab we have provided in mapper.py
  word, count = line.split('\t', 1)
  # convert count (currently a string) to int
  try:
    count = int(count)
  except ValueError:
    # count was not a number, so silently
    # ignore/discard this line
    continue

  # this IF-switch only works because Hadoop sorts map output
  # by key (here: word) before it is passed to the reducer
  if current_word == word:
    current_count += count
  else:
    if current_word: #to not print current_word=None
      # write result to STDOUT
      print('%s\t%s' % (current_word, current_count))
    current_count = count
    current_word = word

# do not forget to output the last word if needed!
if current_word == word:
  print('%s\t%s' % (current_word, current_count))

Writing reducer.py


In [18]:
#Testing our MapReduce job locally (Hadoop does not participate here)
!cat 101.txt | python mapper.py | sort -k1,1 | python reducer.py | head -50
#We apply sorting after the mapper because it is the default operation in MapReduce architecture

#	31
##	2
##!	1
&	10
(.	8
**	4
***	8
*****	2
+	2
+------------+	2
-	39
-----------	1
-------------	1
---------------	1
----------------------	1
----------------------------	1
---------------------------------	1
-----------------------------------	1
.	271
."	9
.'	1
.)	8
:	1
=	2
@	1
_____________________________________	1
~~~~~~~~~~~~~~	1
~~~~~~~~~~~~~~~~~~~~~	2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~	1
($1	1
$10,000	1
$1.2	1
$129.	1
$13,	1
$17,099.	1
$1877.20).	1
$20.	1
$233,000	1
$233,846.14,	1
$233,880	2
$24,639.05--based,	1
$25	1
$250,000--through	1
$252.36.	1
$27	1
$300,000	1
$31	1
$313.	1
$35	1
$367.	1
Traceback (most recent call last):
  File "/content/reducer.py", line 32, in <module>
    print('%s\t%s' % (current_word, current_count))
BrokenPipeError: [Errno 32] Broken pipe


In [19]:
#Changing the permissions of the files
!chmod 777 /content/mapper.py /content/reducer.py
#Setting 777 permissions to a file or directory means that it will be readable, writable and executable by all users

In [20]:

#Running MapReduce programs
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.3.jar \
  -input /word_count_with_python/101.txt \
  -output /word_count_with_python/output \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

packageJobJar: [/tmp/hadoop-unjar5689036370681713477/] [] /tmp/streamjob3430253769660948106.jar tmpDir=null
2024-05-04 07:40:22,046 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-04 07:40:22,632 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-05-04 07:40:23,215 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1714808272884_0002
2024-05-04 07:40:23,736 INFO mapred.FileInputFormat: Total input files to process : 1
2024-05-04 07:40:23,855 INFO mapreduce.JobSubmitter: number of splits:2
2024-05-04 07:40:24,079 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1714808272884_0002
2024-05-04 07:40:24,081 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-05-04 07:40:24,375 INFO conf.Configuration: resource-types.xml not found
2024-05-04 07:40:24,376 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-05-04 07:40:24,4

In [21]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-05-04 07:40 /word_count_with_python/output/_SUCCESS
-rw-r--r--   1 root supergroup     233636 2024-05-04 07:40 /word_count_with_python/output/part-00000


In [22]:
#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count_with_python/output/part-00000 | head -50

"(d)	1
"/H"	1
"0"	1
"02"	2
"1.	1
"414	3
"A	2
"AT&T	3
"AT&T's	1
"Access	1
"Acid	3
"Ad-hocracy"	1
"Advanced	1
"Agents	1
"Al	3
"All	3
"American	1
"An	1
"And	2
"Any	1
"Are	3
"Artificial	1
"As	1
"Assistant	1
"Attctc"	2
"Auld	1
"Autodesk,"	1
"BBS,"	2
"BIRTHPLACE	1
"BRITS	1
"Barry	1
"Because	1
"Before	1
"Bell	2
"Bell"	1
"BellSouth	2
"Berkeley	1
"Big	1
"Biggest	1
"Black	3
"Blue	1
"Bob"	1
"Bob,	3
"Bob:	1
"Bullet-N-Board."	1
"Bureaucrat-ese	1
"But	4
"C-word."	1
"CALIFORNIA"	1
"CC,"	1
cat: Unable to write to output stream.


In [23]:
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /word_count_with_python/output/part-00000 /content/hdfs-wordcount.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')